In [ ]:
# packages

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import csv
import pandas as pd
import transformers
import torch

from torch import cuda, bfloat16
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from sklearn.metrics import classification_report

In [2]:
import gc
gc.collect()
torch.cuda.empty_cache()

# Load the Model from Huggingface

In [5]:
torch.manual_seed(0)

model_id = 'berkeley-nest/Starling-LM-7B-alpha'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()

print(f"Model loaded on {device}")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id)

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.69s/it]


Model loaded on cuda:0


In [6]:
device = torch.device('cuda')
print("GPU Name:", torch.cuda.get_device_name(device))
print("Memory Usage:", torch.cuda.memory_allocated(device) / 1024 ** 3, "GB")
print("Max Memory Usage:", torch.cuda.max_memory_allocated(device) / 1024 ** 3, "GB")

GPU Name: NVIDIA GeForce RTX 3090
Memory Usage: 3.84267520904541 GB
Max Memory Usage: 3.9352827072143555 GB


In [ ]:
torch.manual_seed(0)
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    pad_token_id=tokenizer.eos_token_id,
    temperature=0,  # 'randomness' of outputs, 0.0 is not possible, so use a very small number
    max_new_tokens=3000,  # max number of tokens to generate in the output
    repetition_penalty=1.1  
)

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
def zero_shot_prompt_messages(system_prompt, main_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": main_prompt},
    ]
    prompt = generate_text.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

def count_words(sentences_list):
    # Join the list into a single string
    if sentences_list not in [None, np.nan]:
        full_text = ' '.join(sentences_list)
        # Split the string into words and count them
        word_count = len(full_text.split())
    else:
        word_count = 0
    return word_count

def regex_extract_to_dataframe(strings):
    # Initialize empty lists to store extracted values
    source_names = []
    measures_mentioned_values = []
    supportive_stance_values = []
    critical_stance_values = []

    # Define regex pattern for structured output
    assistant_output_pattern = r'<\|end_of_turn\|>GPT4 Correct Assistant:\s*({.*})'
    
    # Define regex patterns for JSON fields
    source_name_pattern = r'"source_name"\s*:\s*"?([^"]+)"?'
    measures_mentioned_pattern = r'"measures_mentioned"\s*:\s*"?(\d)"?'
    supportive_stance_pattern = r'"supportive_stance"\s*:\s*"?(\d)"?'
    critical_stance_pattern = r'"critical_stance"\s*:\s*"?(\d)"?'

    # Iterate through each string
    for string_data in strings:
        # Extract content after "<|end_of_turn|>GPT4 Correct Assistant:"
        match = re.search(assistant_output_pattern, string_data, re.DOTALL)
        if match:
            json_data = match.group(1).replace("\\", "")  # Extract and clean JSON part
        else:
            continue  # Skip this string if no match found

        # Use regex to find matches for each field
        source_name_match = re.search(source_name_pattern, json_data)
        measures_mentioned_match = re.search(measures_mentioned_pattern, json_data)
        supportive_stance_match = re.search(supportive_stance_pattern, json_data)
        critical_stance_match = re.search(critical_stance_pattern, json_data)

        # Extract values from regex matches
        source_name = source_name_match.group(1) if source_name_match else None
        measures_mentioned = int(measures_mentioned_match.group(1)) if measures_mentioned_match else None
        supportive_stance = int(supportive_stance_match.group(1)) if supportive_stance_match else None
        critical_stance = int(critical_stance_match.group(1)) if critical_stance_match else None

        # Append values to the respective lists
        source_names.append(source_name)
        measures_mentioned_values.append(measures_mentioned)
        supportive_stance_values.append(supportive_stance)
        critical_stance_values.append(critical_stance)

    # Create a DataFrame using the extracted values
    df = pd.DataFrame({
        "source_name": source_names,
        "measures_mentioned": measures_mentioned_values,
        "supportive_stance": supportive_stance_values,
        "critical_stance": critical_stance_values
    })

    return df


# Retrieve Actor DF

In [ ]:
df = pd.read_csv('path_to_test_data_researcher_codings',
                 sep = ';', encoding = 'utf-8', quoting=csv.QUOTE_NONNUMERIC)

# change article_id to integer
df['article_id'] = df['article_id'].astype(int)
# keep only if directly_quoted or indirectly_quoted is 1
df = df[(df['directly_quoted'] == 1) | (df['indirectly_quoted'] == 1)]

In [10]:
# get article_id, actor_name, talks_covid_measures, relevant_sentences_string
df = df[['article_id', 'actor_name', 'talks_covid_measures', 'input_text_corrected',
       'talks_covid_corrected', 'measures_positive_corrected',
       'measures_negative_corrected', 'measures_neutral_corrected']]

In [11]:

# rename actor_name to source_name
df_selected = df.rename(columns={'actor_name': 'source_name'})

# sort by article_id, actor_name
df_selected = df_selected.sort_values(by=['article_id', 'source_name'])

# Does the Actor Talk about Covid Measures? 

In [ ]:
system_prompt = """
You are a helpful AI assistant. Sources in a news article are people or organizations that are quoted in the article. 
You will be provided with the names of these sources and the sentences from the news article where these sources are mentioned. Your task is to determine if the source discusses any COVID-19 measures and, if so, assess the source's overall stance towards these measures.
"""

main_prompt = """
Some examples of COVID-19 Measures:
1. General Coronavirus Measures: Broad discussions about pandemic measures and advice without specifying details.
2. Curfew, Lockdown, Quarantine Rules: References to staying home, lockdowns, quarantines, and related rules.
3. COVID-19 Testing: Discussions about testing requirements and access cards or travel passes related to testing (e.g., Testing for Access (1G)).
4. Vaccination and Procedures: Discussions about vaccines, vaccination processes, side effects, and access cards or travel passes related to vaccines (e.g., 2G or 3G passes).
5. Measures about Sports, Events and Gatherings: Regulations about events, sports, gatherings, and social distancing.
6. Closing of Schools, Restaurants, Shops: References to closures of schools, restaurants, shops, or other public places.
7. Other advice and Recommendations: Wearing face masks, working from home, social distancing, limiting visits, and other health advice.

Read the extracted information from the news article with ID {article_id} about the source: {source_name}. The following text begins with the source name and provides the sentences the source is mentioned in the article: {input_text_corrected}.

1. Carefully review the information provided.

2. Determine if the source mentions any COVID-19 measures:
   - If yes, set measures_mentioned to 1.
   - If no, set measures_mentioned to 0.

3. If measures_mentioned is 1, classify the source's stance towards the COVID-19 measures with two binary flags. 
   - Supportive stance: Set supportive_stance to 1 if the source is positive about the measures, expresses general support or approval towards measures, or generally agrees with the existance of measures.
   - Critical stance: Set critical_stance to 1 the source is negative about the measures,expresses general opposition or criticism towards measures, discusses negative aspects, or disagrees with the existance of  measures.
   - Set supportive_stance and critical_stance to 0 if the source does not mention COVID-19 measures at all or does not express a supportive or critical stance.
   - Keep in mind that the source may express both supportive and critical stances, so both flags can be 1.

4. Provide the result in the following JSON format. Do not include any additional information or explanation.

### Example Output (JSON format):
{{
  "article_id": "2000000",
  "source_name": "Herman Kroneman",
  "measures_mentioned": 1,
  "supportive_stance": 1,
  "critical_stance": 0
}}
{{
  "article_id": "2000001",
  "source_name": "De Volkskrant",
  "measures_mentioned": 0,
  "supportive_stance": 0,
  "critical_stance": 0
}}
{{
  "article_id": "2000002",
  "source_name": "RIVM",
  "measures_mentioned": 1,
  "supportive_stance": 0,
  "critical_stance": 1
}}
"""

In [52]:
zero_shot_prompt = zero_shot_prompt_messages(system_prompt, main_prompt)
print(zero_shot_prompt)

<s>GPT4 Correct System: 
You are a helpful AI assistant. Sources in a news article are people or organizations that are quoted in the article. 
You will be provided with the names of these sources and the sentences from the news article where these sources are mentioned. Your task is to determine if the source discusses any COVID-19 measures and, if so, assess the source's overall stance towards these measures.
<|end_of_turn|>GPT4 Correct User: 
Some examples of COVID-19 Measures:
1. General Coronavirus Measures: Broad discussions about pandemic measures and advice without specifying details.
2. Curfew, Lockdown, Quarantine Rules: References to staying home, lockdowns, quarantines, and related rules.
3. COVID-19 Testing: Discussions about testing requirements and access cards or travel passes related to testing (e.g., Testing for Access (1G)).
4. Vaccination and Procedures: Discussions about vaccines, vaccination processes, side effects, and access cards or travel passes related to vac

In [53]:
prompt_template = PromptTemplate(
    input_variables=["article_id", "source_name", "input_text_corrected"],
    template=zero_shot_prompt
)

In [54]:
chain_one = LLMChain(llm = llm, prompt = prompt_template)
chain_one

LLMChain(prompt=PromptTemplate(input_variables=['article_id', 'input_text_corrected', 'source_name'], template='<s>GPT4 Correct System: \nYou are a helpful AI assistant. Sources in a news article are people or organizations that are quoted in the article. \nYou will be provided with the names of these sources and the sentences from the news article where these sources are mentioned. Your task is to determine if the source discusses any COVID-19 measures and, if so, assess the source\'s overall stance towards these measures.\n<|end_of_turn|>GPT4 Correct User: \nSome examples of COVID-19 Measures:\n1. General Coronavirus Measures: Broad discussions about pandemic measures and advice without specifying details.\n2. Curfew, Lockdown, Quarantine Rules: References to staying home, lockdowns, quarantines, and related rules.\n3. COVID-19 Testing: Discussions about testing requirements and access cards or travel passes related to testing (e.g., Testing for Access (1G)).\n4. Vaccination and Proc

In [55]:
generated_text_zeroshot_actors = []

In [ ]:
%%time
torch.manual_seed(0)
for index, row in df_selected.iterrows():  
    article_id = row['article_id']
    source_name = row['source_name']
    input_text_corrected = row['input_text_corrected']
    input_variables = {
            "article_id": article_id,
            "source_name": source_name,
            "input_text_corrected": input_text_corrected}
    # Generate text using the chain
    generated_text = chain_one.run(input_variables)
    print(generated_text)
    generated_text_zeroshot_actors.append(generated_text)    

# CPU times: total: 15min 37s
# Wall time: 16min 38s

In [ ]:
df_generated = regex_extract_to_dataframe(generated_text_zeroshot_actors)

In [62]:
df_generated.measures_mentioned.value_counts(dropna=False)

measures_mentioned
1    166
0    129
Name: count, dtype: int64

In [63]:
df_generated.supportive_stance.value_counts(dropna=False)

supportive_stance
0    272
1     23
Name: count, dtype: int64

In [64]:
df_generated.critical_stance.value_counts(dropna=False)

critical_stance
0    250
1     45
Name: count, dtype: int64

In [ ]:
# merge the generated data with the original data on index
df_merged = df_selected.merge(df_generated, left_index=True, right_index=True)

In [ ]:
# add mentions_covid_measures to the original dataframe
df_selected['measures_mentioned'] = df_generated['measures_mentioned']
df_selected['supportive_stance'] = df_generated['supportive_stance']
df_selected['critical_stance'] = df_generated['critical_stance']
df_selected['actor_name_starling'] = df_selected['source_name']

In [67]:
# see where source_name and actor_name are different
df_selected[df_selected['source_name'] != df_selected['actor_name_starling']]

,article_id,source_name,talks_covid_measures,input_text_corrected,talks_covid_corrected,measures_positive_corrected,measures_negative_corrected,measures_neutral_corrected,measures_mentioned,supportive_stance,critical_stance,actor_name_starling


In [70]:
# crosstab talks_covid_measures and mentions_covid_measures
pd.crosstab(df_selected['talks_covid_measures'], df_selected['measures_mentioned'])

measures_mentioned,0,1
talks_covid_measures,,
0.0,82,64
1.0,47,102


In [72]:
pd.crosstab(df_selected['measures_positive_corrected'], df_selected['supportive_stance'])

supportive_stance,0,1
measures_positive_corrected,,
0.0,247,8
1.0,25,15


In [73]:
pd.crosstab(df_selected['measures_negative_corrected'], df_selected['critical_stance'])

critical_stance,0,1
measures_negative_corrected,,
0.0,234,32
1.0,16,13


In [74]:
# get classification report
print(classification_report(df_selected['talks_covid_measures'], df_selected['measures_mentioned']))

              precision    recall  f1-score   support

         0.0       0.64      0.56      0.60       146
         1.0       0.61      0.68      0.65       149

    accuracy                           0.62       295
   macro avg       0.63      0.62      0.62       295
weighted avg       0.62      0.62      0.62       295



In [79]:
talks_covid = df_selected[df_selected['talks_covid_measures'] == 1]

In [80]:
print(classification_report(talks_covid['measures_positive_corrected'], talks_covid['supportive_stance']))

              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90       113
         1.0       0.79      0.42      0.55        36

    accuracy                           0.83       149
   macro avg       0.81      0.69      0.72       149
weighted avg       0.83      0.83      0.81       149



In [81]:
print(classification_report(talks_covid['measures_negative_corrected'], talks_covid['critical_stance']))

              precision    recall  f1-score   support

         0.0       0.87      0.86      0.87       121
         1.0       0.43      0.46      0.45        28

    accuracy                           0.79       149
   macro avg       0.65      0.66      0.66       149
weighted avg       0.79      0.79      0.79       149



In [ ]:
# save df
df_selected.to_csv('path_to_save_test_df_with_predictions/stance_Starling.csv',
          sep = ';', encoding = 'utf-8', index = False)